In [2]:
# pip install prophet == 1.1.6
# pip install keras == 3.9.0 tensorflow == 2.19.0
# pip install neuralprophet == 0.8.0
# pytorch-forecasting-1.3.0
#pip install nixtla == 0.6.6
#pip install pmdarima == 2.0.4
#pip install scikeras == 0.13.0
#pip install keras-tcn == 3.1.2

In [2]:
import os
import pickle
import pandas as pd
import numpy as np
import plotly
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns

from itertools import combinations
from sklearn.model_selection import PredefinedSplit, GridSearchCV, TimeSeriesSplit
from joblib import Parallel, delayed
from sklearn.preprocessing import StandardScaler


#timesgpt
from nixtla import NixtlaClient
#xgboost
import xgboost as xgb
from xgboost.callback import EarlyStopping
#lstm
from scikeras.wrappers import KerasRegressor
from tcn import TCN
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Sequential

#from tensorflow.keras.optimizers import Adam
#from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
#from tensorflow.keras.callbacks import EarlyStopping

from xgboost import XGBRegressor
from statsmodels.tsa.arima.model import ARIMA
from prophet import Prophet
from neuralprophet import NeuralProphet

#evaluation
from sklearn.metrics import (
    mean_squared_error, 
    r2_score, 
    mean_absolute_percentage_error,
    mean_absolute_error,
    root_mean_squared_error)

ERROR:NP.plotly:Importing plotly failed. Interactive plots will not work.
ERROR:NP.plotly:Importing plotly failed. Interactive plots will not work.


In [3]:
# Set pandas display options
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', 1000)       # Set display width
pd.set_option('display.max_colwidth', 100) # Show full feature lists
pd.set_option('display.float_format', '{:.4f}'.format)  # 4 decimal places

# If you want to force standard notation (no scientific):
pd.set_option('display.float_format', lambda x: '%.4f' % x if abs(x) > 1e-4 else '%.4e' % x)

In [4]:
def load_dfs_from_folder(folder_path):
    """Loads DataFrames from files in a specified folder and returns a dictionary."""
    dfs = {}
    # List all files in the folder
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".pkl"):
            key = file_name.replace(".pkl", "")  # Extract key from the file name
            file_path = os.path.join(folder_path, file_name)
            
            # Load the dataframe from the pickle file
            with open(file_path, 'rb') as f:
                dfs[key] = pickle.load(f)
            print(f"Loaded {key} from {file_path}")
    
    return dfs

# Load both product_dfs and lagged_product_dfs from their respective folders
product_dfs = load_dfs_from_folder("product_dfs_folder")
lagged_product_dfs = load_dfs_from_folder("lagged_product_dfs_folder")

Loaded P1 from product_dfs_folder\P1.pkl
Loaded P11 from product_dfs_folder\P11.pkl
Loaded P12 from product_dfs_folder\P12.pkl
Loaded P13 from product_dfs_folder\P13.pkl
Loaded P14 from product_dfs_folder\P14.pkl
Loaded P16 from product_dfs_folder\P16.pkl
Loaded P20 from product_dfs_folder\P20.pkl
Loaded P3 from product_dfs_folder\P3.pkl
Loaded P36 from product_dfs_folder\P36.pkl
Loaded P4 from product_dfs_folder\P4.pkl
Loaded P5 from product_dfs_folder\P5.pkl
Loaded P6 from product_dfs_folder\P6.pkl
Loaded P8 from product_dfs_folder\P8.pkl
Loaded P9 from product_dfs_folder\P9.pkl
Loaded Sales_CPI from product_dfs_folder\Sales_CPI.pkl
Loaded P1 from lagged_product_dfs_folder\P1.pkl
Loaded P11 from lagged_product_dfs_folder\P11.pkl
Loaded P12 from lagged_product_dfs_folder\P12.pkl
Loaded P13 from lagged_product_dfs_folder\P13.pkl
Loaded P16 from lagged_product_dfs_folder\P16.pkl
Loaded P20 from lagged_product_dfs_folder\P20.pkl
Loaded P3 from lagged_product_dfs_folder\P3.pkl
Loaded P36 

In [5]:
for product_id in product_dfs.keys():
    product_dfs[product_id] = product_dfs[product_id].rename(columns={product_id: "Sales"})

for product_id in lagged_product_dfs.keys():
    lagged_product_dfs[product_id] = lagged_product_dfs[product_id].rename(columns={product_id: "Sales"})

In [6]:
product_dfs['P1'].head()

,Sales,MAB_ELE_SHP840,PRI27276_org,PRO27826_org,MAB_ELE_PRO276,MAB_ELE_SHP1100
month_year,,,,,,
2018-10-01,35774028.5209,127.8088,109.1196,118.6708,124.2279,130.9893
2018-11-01,5063648.6000,117.6759,109.2248,120.4670,127.4041,132.9341
2018-12-01,37321267.9382,123.2801,109.3301,105.3787,120.5186,131.2613
2019-01-01,27090400.9380,111.0438,109.7510,107.1749,104.7763,113.0576
2019-02-01,34132093.4229,116.7369,109.8562,110.6476,109.5970,117.7047


In [7]:
lagged_product_dfs['P8'].head()

,Sales,PRI27840_org,RohCOPPER1000_org,time_idx,year,month,P8_lag_1,P8_lag_2,P8_lag_5,P8_lag_6,P8_lag_10,P8_ma_1,P8_ma_2,P8_ma_5,P8_ma_6,P8_ma_10
month_year,,,,,,,,,,,,,,,,
2019-08-01,398332.2744,110.6561,75.7745,10,2019,8,368081.2008,420287.0223,582419.0346,361474.5342,580778.2653,398332.2744,383206.7376,380139.7221,413852.9409,400303.6854
2019-09-01,1086855.2918,111.0503,76.4355,11,2019,9,398332.2744,368081.2008,416386.5006,582419.0346,518398.3785,1086855.2918,742593.7831,514233.4804,497925.6504,457149.3767
2019-10-01,112014.9227,111.0322,76.4097,12,2019,10,1086855.2918,398332.2744,297611.6126,416386.5006,267418.3494,112014.9227,599435.1073,477114.1424,447197.0541,441609.0341
2019-11-01,540208.8029,111.0354,77.7720,13,2019,11,112014.9227,1086855.2918,420287.0223,297611.6126,372627.9465,540208.8029,326111.8628,501098.4985,487629.9191,458367.1197
2019-12-01,491332.7200,111.2866,80.6535,14,2019,12,540208.8029,112014.9227,368081.2008,420287.0223,361474.5342,491332.7200,515770.7614,525748.8024,499470.8688,471352.9383


In [8]:
def preprocessing_pipeline(train, val=None, test=None, outlier_treatment=True):

    def winsorize_function(df, cols, lower_quantile=0.01, upper_quantile=0.99):
        """Apply Winsorization only to the product sales column"""
        df = df.copy()
        bounds = {}
        for col in cols:
            q1 = df[col].quantile(lower_quantile)
            q3 = df[col].quantile(upper_quantile)
            df[col] = df[col].clip(lower=q1, upper=q3)
            bounds[col] = (q1, q3)  # Store actual percentile values
            print(f"{col}: Winsorized Bounds -> Lower = {q1:.2f}, Upper = {q3:.2f}")

        return df, bounds
        
    def process_dataset(df, cols, is_train=True, bounds=None):
        df = df.copy() 
        if is_train:
            if outlier_treatment:
                df, bounds = winsorize_function(df, cols)
            else:
                bounds = {}  
        else:
            if outlier_treatment:
                if bounds is None:
                    raise ValueError("Bounds must be provided for validation and test datasets.")
                for col in cols: 
                    if col in bounds:
                        lower, upper = bounds[col]
                        df[col] = df[col].clip(lower, upper)  # Corrected clipping
        return (df, bounds) if is_train else df

    # Process the training dataset
    train, bounds = process_dataset(train, cols = train.columns, is_train=True)

    # Process validation and test datasets with correct bounds
    if val is not None:
        val = process_dataset(val, cols = val.columns, is_train=False, bounds=bounds)

    if test is not None:
        test = process_dataset(test, cols = test.columns, is_train=False, bounds=bounds)
        
    # Return the datasets 
    if test is not None and val is not None:
        return train, val, test
    elif val is not None:
        return train, val
    elif test is not None:
        return train, test
    else:
        return train

def time_series_train_test_split(X, y, test_size=10):
    """Split time series data maintaining temporal order"""
    split_idx = len(X) - test_size
    return (
        X.iloc[:split_idx], X.iloc[split_idx:],
        y.iloc[:split_idx], y.iloc[split_idx:])

def prepare_time_series_data(df_train, feature_set, target_col="Sales", horizon=10, winsorize=False, scaling=False):

    # Extract product data
    data = df_train.copy()
    
    y = data[[target_col]]  
    X = data.drop(columns=[target_col])
    
    # Select features - handle empty feature_set case
    if feature_set:
        try:
            # Convert feature_set to list if it's not already
            if not isinstance(feature_set, list):
                feature_set = [feature_set]
            # Select only columns that exist in X
            available_features = [f for f in feature_set if f in X.columns]
            X = X[available_features]
        except KeyError as e:
            raise ValueError(f"Some features not found in DataFrame: {e}")
    
    # Train/Test Split
    X_train, X_val, y_train, y_val = time_series_train_test_split(X, y, test_size=horizon)

    # Apply preprocessing steps
    y_train, y_val = preprocessing_pipeline(y_train, y_val, test=None, outlier_treatment=winsorize)

    # Scaling
    if scaling: 
        scaler_X = StandardScaler()
        X_train_scaled = scaler_X.fit_transform(X_train)
        X_val_scaled = scaler_X.transform(X_val)
        
        scaler_y = StandardScaler()
        y_train_scaled = scaler_y.fit_transform(y_train)
        y_val_scaled = scaler_y.transform(y_val)
        
        # Convert back to DataFrame for easier handling
        X_train_scaled = pd.DataFrame(X_train_scaled, index=X_train.index, columns=X_train.columns)
        X_val_scaled = pd.DataFrame(X_val_scaled, index=X_val.index, columns=X_val.columns)
        y_train_scaled = pd.DataFrame(y_train_scaled, index=y_train.index, columns=y_train.columns)
        y_val_scaled = pd.DataFrame(y_val_scaled, index=y_val.index, columns=y_val.columns)
        
        return X_train_scaled, X_val_scaled, y_train_scaled, y_val_scaled
    else:
        return X_train, X_val, y_train, y_val

In [9]:
def generate_feature_combinations(features, max_features=None):
    """Generate all possible feature combinations"""
    if max_features is None:
        max_features = len(features)
    
    all_combinations = []
    for r in range(1, max_features + 1):
        all_combinations.extend(combinations(features, r))
    
    return [list(comb) for comb in all_combinations]

In [10]:
def convert_results_to_df(results):
    if not results:
        return pd.DataFrame()  # Retorna um DataFrame vazio se results estiver vazio

    data = []

    # Itera sobre cada dicionário dentro da lista de resultados
    for result in results:
        product_id = result.get('product_id', None)
        winsorize = result.get('winsorize', None)
        
        # Verifica se 'features' existe e se não está vazio
        features = ', '.join(result.get('features', [])) if result.get('features') else 'all'
        
        metrics = result.get('metrics', {})

        validation_predictions = result.get('validation_predictions', [])

        # Adiciona uma linha para cada previsão de validação
        for pred in validation_predictions:
            data.append({
                'product_id': product_id,
                'winsorize': winsorize,
                'features': features,
                'RMSE': metrics.get('RMSE', np.nan),
                'MAPE': metrics.get('MAPE', np.nan),
                'Overfit_Score': metrics.get('Overfit Score', np.nan),
                'validation_prediction': pred
            })
    
    return pd.DataFrame(data)

In [11]:
def calculate_metrics(y_true, y_pred, y_train=None, y_train_pred=None, print_metrics=False):
    metrics = {
        'RMSE': np.sqrt(mean_squared_error(y_true, y_pred)),
        'MAPE': mean_absolute_percentage_error(y_true, y_pred) * 100
    }
    
    if y_train is not None and y_train_pred is not None:
        train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
        metrics['Overfit Score'] = (metrics['RMSE'] - train_rmse) / max(train_rmse, 1e-10)
    
    if print_metrics:
        print("\n=== Metrics ===")
        print(f"RMSE: {metrics['RMSE']:.3f}")
        print(f"MAPE: {metrics['MAPE']:.2f}%")
        if 'Overfit Score' in metrics:
            print(f"Overfit Score: {metrics['Overfit Score']:.3f}")
    
    return metrics

def convert_results_to_df(results):
    """Convert results to DataFrame format"""
    if results is None:
        return pd.DataFrame()
    
    return pd.DataFrame([{
        'product_id': results['product_id'],
        'winsorize': results['winsorize'],
        'features': ', '.join(results['features']) if results['features'] else 'all',
        'RMSE': results['metrics']['RMSE'],
        'MAPE': results['metrics']['MAPE'],
        'Overfit_Score': results['metrics'].get('Overfit Score', np.nan),
        'method': results['method']
    }])

### **4. Modeling Pipeline (Feature selection, Scaling, Model testing):**

#### 4.1. Train vs Validation Splitting Strategy

Our pipeline is going to iterate over:
- **Walk-Forward Validation:** One-step forecasting where the model predicts the next step, and the validation window moves forward one step at a time.



#### 4.2. Feature Combinations

- No macro/lags features (baseline model).
- Single-feature models.
- Multi-feature models.

#### 4.4. Models

We will tune hyperparameters for all the models for each product, with the best features.


**Time Series Model Comparison Table**

| Feature               | TimeGPT (Zero-Shot) | XGBoost            | TCN                | ARIMA              | Neural Prophet     |
|-----------------------|---------------------|--------------------|--------------------|--------------------|--------------------|
| **Scaling Needed?**   | ❌ No               | ❌ No              | ✅ Yes             | ❌ No (if stationary) | ✅ Yes            |
| **Handles Trends**    | ✅ Excellent        | ✅ (With features) | ✅ Excellent       | ✅ (With differencing) | ✅ Excellent     |
| **Handles Seasonality** | ✅ Automatic       | ❌ (Requires manual lags) | ✅ (With large receptive field) | ✅ (SARIMA) | ✅ Automatic      |
| **Multivariate Support** | ✅ Yes            | ✅ Yes             | ✅ Yes             | ❌ Univariate only | ✅ Yes             |
| **Training Speed**    | ⚡ Instant (pre-trained) | 🏎️ Fast          | 🐢 Slow (GPU helps) | 🏃 Moderate        | 🐢 Slow           |
| **Interpretability**  | ❌ Black box        | ✅ Feature importances | ❌ Black box     | ✅ Model coefficients | ✅ Component plots |
| **Best For**          | Zero-shot forecasting | Tabular data with temporal features | Long-range dependencies | Simple univariate series | Hybrid (AR + NN) patterns |
| **Weakness**          | Limited control     | Poor with long sequences | Computationally heavy | Linear assumptions only | Overfit risk       |


1. **TimeGPT** (by Nixtla):
   - Pros: No training needed, handles any frequency, great for quick benchmarks.
   - Cons: Proprietary, limited customization.

2. **XGBoost**:
   - Best when you have:
     - Easy and fast. 

3. **TCN** (Temporal Convolutional Networks):
   - Pros: Captures long-range patterns better than LSTMs.
   - Cons: Needs careful tuning of and is more computationaly expensive.

4. **ARIMA**:
   - Important:
     - Data is stationary (or differenced) 
     - Short-term forecasts (≤12 steps)
   - Warning: Fails miserably with:
     - Non-linear trends
     - High-frequency data, wich is ok, bc we are dealing with months.

5. **Neural Prophet**:
   - Hybrid of AR terms and neural nets.
   - Best when: needed interpretable components (trend, seasonality)


Summary:

- **We will Scale**: TCN, Neural Prophet
- **Optional Scaling**: XGBoost (for extreme values)
- **We will not Scale**: ARIMA (but difference if non-stationary!), TimeGPT

1. We will start with **TimeGPT** for a baseline.
2. Then test **XGBoost** with lag features.
3. Then **TCN** to see he effect for more complex patterns.
4. **Neural Prophet** and **ARIMA** for interpretability.

#### 4.5. Evaluation Metrics: 
- RMSE → Measures error magnitude.
- MAPE → Percentage-based error, useful for business impact.
- R² → Measures how well the model explains variance.
- Overfit Score →  

$$
\text{Overfit Score} = \frac{\text{Test RMSE} - \text{Train RMSE}}{\text{Train RMSE}}
$$



    - < 0.1 (Underfit)
    - 0.1 - 0.5 (Good Fit)
    - > 0.5 (Overfit Warning!)

#### Modeling functions

In [ ]:
def plot_model_performance(model, X_train, y_train, X_test, y_test, 
                          title="Model Performance", show_confidence=True):
    # Get predictions
    train_pred = model.predict(X_train)
    test_pred = model.predict(X_test)
    
    # Create figure with subplots
    fig = make_subplots(rows=2, cols=1, 
                        subplot_titles=("Training Set Performance", "Test Set Performance"),
                        vertical_spacing=0.15)
    
    # Training set plot
    fig.add_trace(go.Scatter(
        x=X_train.index,
        y=y_train,
        mode='lines',
        name='Actual (Train)',
        line=dict(color='black', width=2)
    ), row=1, col=1)
    
    fig.add_trace(go.Scatter(
        x=X_train.index,
        y=train_pred,
        mode='lines',
        name='Predicted (Train)',
        line=dict(color='blue', width=1, dash='dash')
    ), row=1, col=1)
    
    # Test set plot
    fig.add_trace(go.Scatter(
        x=X_test.index,
        y=y_test,
        mode='lines',
        name='Actual (Test)',
        line=dict(color='black', width=2),
        showlegend=False
    ), row=2, col=1)
    
    fig.add_trace(go.Scatter(
        x=X_test.index,
        y=test_pred,
        mode='lines',
        name='Predicted (Test)',
        line=dict(color='red', width=1, dash='dash'),
        showlegend=False
    ), row=2, col=1)
    
    # Add confidence intervals if available and requested
    if show_confidence and hasattr(model, 'predict_interval'):
        try:
            _, train_lower, train_upper = model.predict_interval(X_train)
            fig.add_trace(go.Scatter(
                x=list(X_train.index) + list(X_train.index[::-1]),
                y=list(train_upper) + list(train_lower[::-1]),
                fill='toself',
                fillcolor='rgba(0,100,80,0.2)',
                line=dict(color='rgba(255,255,255,0)'),
                name='95% Confidence',
                showlegend=False
            ), row=1, col=1)
            
            _, test_lower, test_upper = model.predict_interval(X_test)
            fig.add_trace(go.Scatter(
                x=list(X_test.index) + list(X_test.index[::-1]),
                y=list(test_upper) + list(test_lower[::-1]),
                fill='toself',
                fillcolor='rgba(100,0,80,0.2)',
                line=dict(color='rgba(255,255,255,0)'),
                name='95% Confidence',
                showlegend=False
            ), row=2, col=1)
        except:
            pass
    
    # Update layout
    fig.update_layout(
        title=title,
        height=800,
        template="plotly_white",
        hovermode="x unified",
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
    )
    
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Value", row=1, col=1)
    fig.update_yaxes(title_text="Value", row=2, col=1)
    
    fig.show()

In [ ]:
def plot_training_progress(history):
    """For models that provide training history (like neural networks)"""
    if not hasattr(history, 'history'):
        print("Model doesn't have training history to plot")
        return
    
    fig = go.Figure()
    
    # Plot training loss
    fig.add_trace(go.Scatter(
        x=np.arange(len(history.history['loss'])),
        y=history.history['loss'],
        mode='lines',
        name='Training Loss',
        line=dict(color='blue')
    ))
    
    # Plot validation loss if available
    if 'val_loss' in history.history:
        fig.add_trace(go.Scatter(
            x=np.arange(len(history.history['val_loss'])),
            y=history.history['val_loss'],
            mode='lines',
            name='Validation Loss',
            line=dict(color='orange')
        ))
    
    fig.update_layout(
        title="Training Progress",
        xaxis_title="Epoch",
        yaxis_title="Loss",
        template="plotly_white"
    )
    
    fig.show()

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import pandas as pd
from sklearn.metrics import (
    mean_squared_error, 
    r2_score, 
    mean_absolute_percentage_error,
    mean_absolute_error
)

def calculate_metrics(y_true, y_pred):
    """Calculate all relevant metrics for time series forecasting"""
    metrics = {
        'RMSE': mean_squared_error(y_true, y_pred, squared=False),
        'MAE': mean_absolute_error(y_true, y_pred),
        'MAPE': mean_absolute_percentage_error(y_true, y_pred) * 100,  # as percentage
        'R2': r2_score(y_true, y_pred),
        'Error_Std': np.std(y_true - y_pred)
    }
    return metrics

def print_metrics(metrics, title="Model Performance"):
    """Print metrics in a consistent format"""
    print(f"\n=== {title} ===")
    print(f"RMSE: {metrics['RMSE']:.3f}")
    print(f"MAE: {metrics['MAE']:.3f}")
    print(f"MAPE: {metrics['MAPE']:.2f}%")
    print(f"R²: {metrics['R2']:.3f}")
    print(f"Error Std: {metrics['Error_Std']:.3f}")

def plot_time_series_forecast(
    train_series, 
    test_series, 
    predictions,
    train_predictions=None,
    confidence_intervals=None,
    title="Forecast Performance",
    xaxis_title="Date",
    yaxis_title="Value"
):
    """
    Create interactive forecast visualization with Plotly
    
    Parameters:
    -----------
    train_series : pd.Series with training data
    test_series : pd.Series with actual test values
    predictions : array-like with predicted values
    train_predictions : array-like with predictions on training set (optional)
    confidence_intervals : tuple of (upper_bound, lower_bound) arrays
    """
    fig = go.Figure()
    
    # Training data
    fig.add_trace(go.Scatter(
        x=train_series.index,
        y=train_series,
        mode='lines',
        name='Training Data',
        line=dict(color='#1f77b4', width=2)
    ))
    
    # Training predictions if provided
    if train_predictions is not None:
        fig.add_trace(go.Scatter(
            x=train_series.index,
            y=train_predictions,
            mode='lines',
            name='Training Predictions',
            line=dict(color='#ff7f0e', width=1.5, dash='dot')
        ))
    
    # Test data
    fig.add_trace(go.Scatter(
        x=test_series.index,
        y=test_series,
        mode='lines',
        name='Actual Values',
        line=dict(color='#2ca02c', width=2)
    ))
    
    # Predictions
    fig.add_trace(go.Scatter(
        x=test_series.index,
        y=predictions,
        mode='lines',
        name='Predictions',
        line=dict(color='#d62728', width=2, dash='dash')
    ))
    
    # Confidence intervals if provided
    if confidence_intervals is not None:
        upper, lower = confidence_intervals
        fig.add_trace(go.Scatter(
            x=list(test_series.index) + list(test_series.index[::-1]),
            y=list(upper) + list(lower[::-1]),
            fill='toself',
            fillcolor='rgba(173, 216, 230, 0.2)',
            line=dict(color='rgba(255,255,255,0)'),
            name='95% Confidence',
            showlegend=True
        ))
    
    # Update layout
    fig.update_layout(
        title=title,
        xaxis_title=xaxis_title,
        yaxis_title=yaxis_title,
        template="plotly_white",
        hovermode="x unified",
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        ),
        margin=dict(l=20, r=20, t=40, b=20)
    )
    
    return fig

def plot_training_history(
    history,
    metrics=['loss'],
    val_metrics=None,
    title="Training Progress"
):
    """
    Plot training history for models that provide it (like neural networks)
    
    Parameters:
    -----------
    history : keras History object or dict with training metrics
    metrics : list of metrics to plot from training
    val_metrics : list of corresponding validation metrics
    """
    if not hasattr(history, 'history') and not isinstance(history, dict):
        print("No training history available to plot")
        return None
    
    history_dict = history.history if hasattr(history, 'history') else history
    
    fig = go.Figure()
    
    # Plot training metrics
    for metric in metrics:
        if metric in history_dict:
            fig.add_trace(go.Scatter(
                x=np.arange(1, len(history_dict[metric])+1),
                y=history_dict[metric],
                mode='lines+markers',
                name=f'Training {metric}',
                line=dict(width=2)
            ))
    
    # Plot validation metrics if available
    if val_metrics:
        for val_metric in val_metrics:
            if val_metric in history_dict:
                fig.add_trace(go.Scatter(
                    x=np.arange(1, len(history_dict[val_metric])+1),
                    y=history_dict[val_metric],
                    mode='lines+markers',
                    name=f'Validation {val_metric}',
                    line=dict(width=2, dash='dash')
                ))
    
    fig.update_layout(
        title=title,
        xaxis_title="Epoch",
        yaxis_title="Metric Value",
        template="plotly_white",
        hovermode="x unified"
    )
    
    return fig

def evaluate_and_visualize(
    model,
    X_train, y_train,
    X_test, y_test,
    model_type='standard',
    confidence_intervals=None,
    training_history=None
):
    """
    Comprehensive evaluation and visualization wrapper
    
    Parameters:
    -----------
    model : fitted model
    X_train, y_train : training data
    X_test, y_test : test data
    model_type : str - 'standard', 'sequential', 'prophet', etc.
    confidence_intervals : tuple of (upper, lower) bounds
    training_history : history object for models that track training
    """
    # Get predictions based on model type
    if model_type == 'sequential':  # For Keras models
        train_pred = model.predict(X_train).flatten()
        test_pred = model.predict(X_test).flatten()
    elif model_type == 'prophet':
        train_pred = model.predict(X_train)['yhat'].values
        test_pred = model.predict(X_test)['yhat'].values
    else:  # Standard sklearn-style models
        train_pred = model.predict(X_train)
        test_pred = model.predict(X_test)
    
    # Calculate metrics
    train_metrics = calculate_metrics(y_train, train_pred)
    test_metrics = calculate_metrics(y_test, test_pred)
    
    # Print metrics
    print_metrics(train_metrics, "Training Set Performance")
    print_metrics(test_metrics, "Test Set Performance")
    
    # Create visualizations
    forecast_fig = plot_time_series_forecast(
        train_series=y_train,
        test_series=y_test,
        predictions=test_pred,
        train_predictions=train_pred,
        confidence_intervals=confidence_intervals,
        title="Model Forecast Performance"
    )
    
    if training_history is not None:
        history_fig = plot_training_history(training_history)
        return {
            'metrics': {'train': train_metrics, 'test': test_metrics},
            'forecast_plot': forecast_fig,
            'training_plot': history_fig
        }
    
    return {
        'metrics': {'train': train_metrics, 'test': test_metrics},
        'forecast_plot': forecast_fig
    }

#### 1. TimeGPT

In [ ]:
nixtla_client = NixtlaClient(
    api_key = "nixak-CIwSKQ0cRLIuFR1TYllLFVakTGx3WCY30YPEKfxG0lDQcE0akGo3GE4aMJO9XXbkKjdFaGDP5x6uSxQ6"
)
nixtla_client.validate_api_key()

INFO:nixtla.nixtla_client:Happy Forecasting! :)


True

In [ ]:
def choose_parameters_timegpt(product_id, df_train, feature_set, target_col="Sales", 
                              nixtla_client=None, winsorize=False, 
                              horizon=7):
    
    # --- Input Validation ---
    if nixtla_client is None:
        raise ValueError("NixtlaClient instance is required.")
    if len(df_train) < 12:
        print(f"⚠️ Skipping {product_id} - only {len(df_train)} observations")
        return None
    
    # --- Data Preparation ---
    print('Preparing data...')
    try:
        X_train, X_val, y_train, y_val = prepare_time_series_data(df_train, feature_set, target_col, horizon, winsorize)

    except Exception as e:
        print(f"Data preparation failed: {str(e)}")
        return None
 
    # --- Determine Feature Types ---
    hist_exog = [f for f in feature_set if '_lag_' in f or '_ma_' in f] if feature_set else []
    fut_exog = [f for f in feature_set if f not in hist_exog] if feature_set else []


    # --- Prepare Future Exogenous Features ---
    X_future = None
    if fut_exog:
        X_future = X_val[fut_exog].copy()
        if not X_future.isnull().all().all():  
            X_future.insert(0, 'month_year', X_val.index)
        else:
            X_future = None  # Avoid passing empty DataFrame

    # --- Simple Forecast (12 ≤ obs < 36) ---
    if len(X_train) < 36:
        print(f"Using simple forecast ({len(X_train)} < 36 obs)")
        try:
            history = df_train.copy()
            if feature_set:
                history = history[feature_set + [target_col]]

            forecast = nixtla_client.forecast(df=history.reset_index(),
                                              time_col="month_year", target_col=target_col, h=horizon,
                                              X_df=X_future,  # Always pass X_future if available
                                              hist_exog_list=hist_exog if hist_exog else None)
            
            # Get training predictions
            train_pred = nixtla_client.forecast(df=history.reset_index(),time_col="month_year",
                                                target_col=target_col,h=1,hist_exog_list=hist_exog if hist_exog else None
                                                ).iloc[0]["TimeGPT"]
            
            train_predictions = [train_pred] * len(history)
            
            return {
                'product_id': product_id,'winsorize': winsorize,'features': feature_set,
                'metrics': calculate_metrics(y_true=y_val,y_pred=forecast["TimeGPT"],
                                             y_train=history[target_col],y_train_pred=train_predictions),
                'method': 'simple','validation_predictions': forecast["TimeGPT"]}
        except Exception as e:
            print(f"Simple forecast failed: {str(e)}")
            return None


    # --- Walk-Forward (≥36 obs) ---
    print(f"Using walk-forward ({len(X_train)} ≥ 36 obs)")
    predictions = []
    history = X_train.copy()
    history[target_col] = y_train.values
    history = history.reset_index()
    
    # Get training predictions
    try:
        train_fit = nixtla_client.forecast(df=history,time_col='month_year',target_col=target_col,h=1,
                                           hist_exog_list=hist_exog if hist_exog else None)
        train_predictions = [train_fit["TimeGPT"].iloc[0]] * len(y_train)
    except Exception as e:
        print(f"Training pred failed: {str(e)} - using last value")
        train_predictions = [y_train.iloc[-1]] * len(y_train)

    for i in range(horizon):
        try:
            X_future_step = None
            if fut_exog:
                X_future_step = X_val.iloc[[i]][fut_exog].copy()
                if not X_future_step.isnull().all().all():  
                    X_future_step.insert(0, 'month_year', X_val.index[i])
                else:
                    X_future_step = None  

            forecast = nixtla_client.forecast(df=history,time_col='month_year',target_col=target_col,
                                              h=1, X_df=X_future_step if fut_exog else None, # FUTURE features (macro)
                                              hist_exog_list=hist_exog if hist_exog else None)  # HISTORICAL features (lags, moving averages)
            
            pred = forecast["TimeGPT"].iloc[0]
            predictions.append(pred)
            
            new_row = X_val.iloc[i].copy()
            new_row['month_year'] = X_val.index[i]
            new_row[target_col] = pred
            history = pd.concat([history, pd.DataFrame([new_row])])
        
        except Exception as e:
            print(f"Iteration {i+1} failed: {str(e)}")
            raise
    
    metrics = calculate_metrics(y_true=y_val,y_pred=predictions,y_train=y_train,y_train_pred=train_predictions)
    
    return {'product_id': product_id,'winsorize': winsorize,'features': feature_set,'metrics': metrics,
            'method': 'walkforward','validation_predictions': predictions}


def find_best_timegpt_config(product_id, df_train, nixtla_client, lagged_df=None, target_col="Sales"):

    # Validate input data
    if not isinstance(df_train.index, pd.DatetimeIndex):
        raise TypeError("❌ df_train index is not a DatetimeIndex!")
    if df_train.index.isnull().any():
        raise ValueError("❌ Some index values could not be parsed into datetime!")
    
    results = []
    data = df_train.copy()
    
    # Feature groups for original data
    normal_features = [col for col in data.columns if col != target_col]
    normal_feature_combinations = generate_feature_combinations(normal_features, max_features=6)  
    normal_feature_combinations = [[]] + normal_feature_combinations
    print(normal_feature_combinations)
    
    # Test original data configurations
    for winsorize in [True, False]:
        for features in normal_feature_combinations:
            try:
                print(f"\nTesting - Winsorize: {winsorize}, Features: {features}")

                result = choose_parameters_timegpt(
                    product_id=product_id,df_train=data, feature_set=features, target_col=target_col,
                    nixtla_client=nixtla_client,winsorize=winsorize)
                
                if result:
                    print(f"✅ Success - RMSE: {result['metrics']['RMSE']:.2f}")
                    results.append(result)
            except Exception as e:
                print(f"⚠️ Configuration failed: {str(e)}")
                continue
    
    # Test lagged data configurations if available
    if lagged_df is not None:
        lagged_data = lagged_df.copy()
        lag_features = [col for col in lagged_data.columns if 'lag' in col]
        ma_features = [col for col in lagged_data.columns if '_ma_' in col]
        macro_features = [col for col in data.columns if col != target_col]
        
        # Generate lag feature combinations
        lag_feature_combinations = generate_feature_combinations(lag_features, max_features=5)
        
        # Create combined feature sets
        full_combinations = []
        for lag_combo in lag_feature_combinations:
            full_combinations.append(lag_combo)  # Just lags
            full_combinations.append(lag_combo + macro_features)  # Lags + macros
            full_combinations.append(lag_combo + ma_features)  # Lags + MAs
            full_combinations.append(lag_combo + macro_features + ma_features)  # All
        
        # Remove duplicates
        seen = set()
        unique_combinations = []
        for combo in full_combinations:
            combo_tuple = tuple(sorted(combo))
            if combo_tuple not in seen:
                seen.add(combo_tuple)
                unique_combinations.append(combo)
        
        # Test all unique combinations
        for winsorize in [True, False]:
            for features in unique_combinations:
                try:
                    print(f"\nTesting - Lagged Features: {features}")

                    result = choose_parameters_timegpt(product_id=product_id,df_train=lagged_data,feature_set=features,
                                                       target_col=target_col,nixtla_client=nixtla_client,winsorize=winsorize)
                    
                    if result:
                        print(f"✅ Success - RMSE: {result['metrics']['RMSE']:.2f}")
                        results.append(result)
                except Exception as e:
                    print(f"⚠️ Lagged configuration failed: {str(e)}")
                    continue
    
    if not results:
        raise ValueError(f"No valid configurations for {product_id}")
    
    # Select best configuration by RMSE
    best_config = min(results, key=lambda x: x['metrics']['RMSE'])
    
    print(f"\n✅ Best configuration for {product_id}:")
    print(f"- Winsorize: {best_config['winsorize']}")
    print(f"- Features: {best_config['features']}")
    print(f"- RMSE: {best_config['metrics']['RMSE']:.2f}")
    print(f"- Method: {best_config.get('method', 'walkforward')}")
    
    return best_config, results

def process_product_parallel(product_id):
    try:
        df_train = product_dfs[product_id].rename(columns={product_id: "Sales"})
        
        # Ensure DatetimeIndex
        if not isinstance(df_train.index, pd.DatetimeIndex):
            df_train.index = pd.to_datetime(df_train.index, errors="coerce")
        
        if df_train.index.isnull().any():
            raise ValueError("❌ Some index values could not be parsed into datetime!")

        lagged_df = lagged_product_dfs.get(product_id, None)

        best_config, results = find_best_timegpt_config(product_id=product_id,df_train=df_train,lagged_df=lagged_df,
            nixtla_client=nixtla_client,target_col="Sales")
        
        return best_config, results
    except Exception as e:
        print(f"⚠️ Failed to process {product_id}: {str(e)}")
        return None

#### 3. XGboost

In [ ]:
def choose_parameters_xgboost(product_id, df_train, feature_set, target_col="Sales", 
                            param_grid=None, n_jobs=-1, winsorize=False, horizon=7):
    """Optimized XGBoost forecasting with proper walk-forward validation"""
    
    # --- Input Validation ---
    if len(df_train) < 12:
        print(f"⚠️ Skipping {product_id} - only {len(df_train)} observations")
        return None
    
    # --- Data Preparation ---
    print('Preparing data...')
    try:
        X_train, X_val, y_train, y_val = prepare_time_series_data(
            df_train, feature_set, target_col, horizon, winsorize)
        # Verify temporal ordering
        assert X_train.index[-1] < X_val.index[0], "Validation data must be after training data"
    except Exception as e:
        print(f"Data preparation failed: {str(e)}")
        return None

    # --- Optimized Parameter Grid for Small Data ---
    if param_grid is None:
        param_grid = {
            'n_estimators': [50, 100],
            'max_depth': [2, 3],
            'learning_rate': [0.01, 0.05, 0.1],
            'subsample': [0.9, 1.0],
            'colsample_bytree': [0.9, 1.0],
            'min_child_weight': [1, 3]}

    # --- Efficient Model Setup ---
    base_model = xgb.XGBRegressor(
        objective='reg:squarederror',random_state=42,n_jobs=1,tree_method='hist',  # Faster training
        enable_categorical=False,gamma=0,reg_alpha=0,reg_lambda=1)

    # --- Strict Temporal Validation ---
    test_fold = np.array([-1] * len(X_train) + [0] * len(X_val))
    ps = PredefinedSplit(test_fold)

    # --- Focused Grid Search ---
    grid_search = GridSearchCV(
        estimator=base_model,param_grid=param_grid,scoring='neg_root_mean_squared_error',cv=ps,n_jobs=n_jobs,verbose=1,refit=True)

    # --- Training with Validation ---
    X_combined = pd.concat([X_train, X_val])
    y_combined = pd.concat([y_train, y_val])
    
    print(f"Starting grid search for {product_id}...")
    try:
        grid_search.fit(X_combined, y_combined)
        best_model = grid_search.best_estimator_
    except Exception as e:
        print(f"Grid search failed: {str(e)}")
        return None

    # --- Robust Walk-forward Validation ---
    predictions = []
    print(f"Using walk-forward (n_train={len(X_train)}, horizon={horizon})")
    
    for i in range(horizon):
        try:
            # Create expanding window
            window_X = pd.concat([X_train, X_val.iloc[:i]])
            window_y = pd.concat([y_train, y_val.iloc[:i]])
            
            # Correct XGBoost 2.1.3+ fitting
            best_model.fit(window_X, window_y, eval_set=[(X_val.iloc[[i]], y_val.iloc[[i]])], verbose=0)
    
            pred = best_model.predict(X_val.iloc[[i]]).item()
            predictions.append(pred)
            
        except Exception as e:
            print(f"Iteration {i+1} failed: {str(e)} - using fallback")
            fallback = (predictions[-1] if len(predictions) > 0 else 
                       y_train.iloc[-1] if len(y_train) > 0 else np.nan)
            predictions.append(fallback)

    # --- Training Metrics ---
    try:
        train_predictions = best_model.predict(X_train).tolist()
    except Exception as e:
        print(f"Training predictions failed: {str(e)} - using last value")
        train_predictions = [y_train.iloc[-1]] * len(y_train)

    # --- Comprehensive Metrics ---
    metrics = calculate_metrics(y_true=y_val,y_pred=predictions,y_train=y_train,y_train_pred=train_predictions)
    
    return {
        'product_id': product_id,
        'winsorize': winsorize,
        'features': feature_set,
        'metrics': metrics,
        'method': 'walkforward',
        'best_params': grid_search.best_params_,
        'validation_predictions': predictions,
        'feature_importances': dict(zip(feature_set, best_model.feature_importances_))}


def find_best_xgboost_config(product_id, df_train, lagged_df=None, target_col="Sales", 
                            n_jobs=-1, custom_param_grids=None):
    # Validate input data
    if not isinstance(df_train.index, pd.DatetimeIndex):
        raise TypeError("❌ df_train index is not a DatetimeIndex!")
    if df_train.index.isnull().any():
        raise ValueError("❌ Some index values could not be parsed into datetime!")
    
    results = []
    data = df_train.copy()
    
    # Feature groups for original data
    normal_features = [col for col in data.columns if col != target_col]
    normal_feature_combinations = generate_feature_combinations(normal_features, max_features=6)  
    normal_feature_combinations = [[]] + normal_feature_combinations
    
    # Test original data configurations
    for winsorize in [True, False]:
        for features in normal_feature_combinations:
            try:
                print(f"\nTesting - Winsorize: {winsorize}, Features: {features}")

                # Use custom param grid if provided for this feature set
                param_grid = None
                if custom_param_grids and tuple(features) in custom_param_grids:
                    param_grid = custom_param_grids[tuple(features)]

                result = choose_parameters_xgboost(product_id=product_id,df_train=data,feature_set=features,
                    target_col=target_col,winsorize=winsorize,n_jobs=n_jobs,param_grid=param_grid)
                
                if result:
                    print(f"✅ Success - RMSE: {result['metrics']['RMSE']:.2f}")
                    results.append(result)
            except Exception as e:
                print(f"⚠️ Configuration failed: {str(e)}")
                continue
    
    # Test lagged data configurations if available
    if lagged_df is not None:
        lagged_data = lagged_df.copy()
        lag_features = [col for col in lagged_data.columns if 'lag' in col]
        ma_features = [col for col in lagged_data.columns if '_ma_' in col]
        macro_features = [col for col in data.columns if col != target_col]
        
        # Generate lag feature combinations
        lag_feature_combinations = generate_feature_combinations(lag_features, max_features=5)
        
        # Create combined feature sets
        full_combinations = []
        for lag_combo in lag_feature_combinations:
            full_combinations.append(lag_combo)  # Just lags
            full_combinations.append(lag_combo + macro_features)  # Lags + macros
            full_combinations.append(lag_combo + ma_features)  # Lags + MAs
            full_combinations.append(lag_combo + macro_features + ma_features)  # All
        
        # Remove duplicates
        seen = set()
        unique_combinations = []
        for combo in full_combinations:
            combo_tuple = tuple(sorted(combo))
            if combo_tuple not in seen:
                seen.add(combo_tuple)
                unique_combinations.append(combo)
        
        # Test all unique combinations
        for winsorize in [True, False]:
            for features in unique_combinations:
                try:
                    print(f"\nTesting - Lagged Features: {features}")

                    # Use custom param grid if provided for this feature set
                    param_grid = None
                    if custom_param_grids and tuple(features) in custom_param_grids:
                        param_grid = custom_param_grids[tuple(features)]

                    result = choose_parameters_xgboost(product_id=product_id,df_train=lagged_data,feature_set=features,
                                                       target_col=target_col,winsorize=winsorize,n_jobs=n_jobs,param_grid=param_grid)
                    
                    if result:
                        print(f"✅ Success - RMSE: {result['metrics']['RMSE']:.2f}")
                        results.append(result)
                except Exception as e:
                    print(f"⚠️ Lagged configuration failed: {str(e)}")
                    continue
    
    if not results:
        raise ValueError(f"No valid configurations for {product_id}")
    
    # Select best configuration by RMSE
    best_config = min(results, key=lambda x: x['metrics']['RMSE'])
    
    print(f"\n✅ Best configuration for {product_id}:")
    print(f"- Winsorize: {best_config['winsorize']}")
    print(f"- Features: {best_config['features']}")
    print(f"- Best params: {best_config['best_params']}")
    print(f"- RMSE: {best_config['metrics']['RMSE']:.2f}")
    print(f"- Method: {best_config.get('method', 'walkforward')}")
    
    return best_config, results

def process_product_parallel(product_id, custom_param_grids=None):
    try:
        df_train = product_dfs[product_id].rename(columns={product_id: "Sales"})
        
        # Ensure DatetimeIndex
        if not isinstance(df_train.index, pd.DatetimeIndex):
            df_train.index = pd.to_datetime(df_train.index, errors="coerce")
        
        if df_train.index.isnull().any():
            raise ValueError("❌ Some index values could not be parsed into datetime!")

        lagged_df = lagged_product_dfs.get(product_id, None)

        best_config, results = find_best_xgboost_config(product_id=product_id,df_train=df_train,lagged_df=lagged_df,target_col="Sales", custom_param_grids=custom_param_grids)
        
        return best_config, results
    except Exception as e:
        print(f"⚠️ Failed to process {product_id}: {str(e)}")
        return None

In [ ]:
all_product_ids = set(product_dfs.keys())
all_results_xgboost = []  # List to store all results
best_configs_xgboost = []  # List to store best configurations

for product_id in all_product_ids:
    print(f"Processing product: {product_id}")
    
    try:
        best_config_xgboost, results_xgboost = process_product_parallel(product_id, custom_param_grids=None)
        
        if best_config_xgboost is not None:
            best_configs_xgboost.append(best_config_xgboost)  # Store best config
        else:
            print(f"Warning: No best config returned for product {product_id}")
        
        if results_xgboost is not None:
            all_results_xgboost.extend(results_xgboost)  # Store results
        else:
            print(f"Warning: No results returned for product {product_id}")
            
    except Exception as e:
        print(f"Error processing product {product_id}: {str(e)}")


Processing product: P5

Testing - Winsorize: True, Features: []
Preparing data...
Sales: Winsorized Bounds -> Lower = 1789375.78, Upper = 19056280.70
Starting grid search for P5...
Fitting 1 folds for each of 96 candidates, totalling 96 fits
Using walk-forward (n_train=36, horizon=7)
✅ Success - RMSE: 2125718.32

Testing - Winsorize: True, Features: ['PRO27826_org']
Preparing data...
Sales: Winsorized Bounds -> Lower = 1789375.78, Upper = 19056280.70
Starting grid search for P5...
Fitting 1 folds for each of 96 candidates, totalling 96 fits
Using walk-forward (n_train=36, horizon=7)
✅ Success - RMSE: 1413065.22

Testing - Winsorize: True, Features: ['MAB_ELE_SHP840']
Preparing data...
Sales: Winsorized Bounds -> Lower = 1789375.78, Upper = 19056280.70
Starting grid search for P5...
Fitting 1 folds for each of 96 candidates, totalling 96 fits
Using walk-forward (n_train=36, horizon=7)
✅ Success - RMSE: 2430260.13

Testing - Winsorize: True, Features: ['PRO271000_org']
Preparing data...


KeyboardInterrupt: 

In [ ]:
df_all_results_xgboost = convert_results_to_df(all_results_xgboost) if all_results_xgboost else pd.DataFrame()
df_best_configs_xgboost = pd.DataFrame(best_configs_xgboost) if best_configs_xgboost else pd.DataFrame()

df_all_results_xgboost.to_csv("xgboost_results.csv", index=False)
df_best_configs_xgboost.to_csv("xgboost_best_configs_details.csv", index=False)

print("Processing completed! Results saved.")

Processing completed! Results saved.


#### 4. TCN

In [40]:
def choose_parameters_tcn(product_id, df_train, feature_set, target_col="Sales", 
                         param_grid=None, n_jobs=-1, winsorize=False, horizon=7):
    """Optimized TCN forecasting with walk-forward validation"""
    
    if len(df_train) < 12:
        print(f"⚠️ Skipping {product_id} - only {len(df_train)} observations")
        return None
    
    print('Preparing data...')
    try:
        X_train, X_val, y_train, y_val = prepare_time_series_data(
            df_train, feature_set, target_col, horizon, winsorize, scaling=True)
        
        assert X_train.index[-1] < X_val.index[0], "Validation data must be after training data"
    except Exception as e:
        print(f"Data preparation failed: {str(e)}")
        return None

    # Reshape data for TCN input (samples, timesteps, features)
    X_train = X_train.values.reshape((X_train.shape[0], 1, X_train.shape[1]))
    X_val = X_val.values.reshape((X_val.shape[0], 1, X_val.shape[1]))
    y_train = y_train.values.reshape(-1, 1)
    y_val = y_val.values.reshape(-1, 1)

    if param_grid is None:
        param_grid = {
            'nb_filters': [16, 32],
            'kernel_size': [2, 3],
            'dilations': [[1, 2, 4], [1, 2, 4, 8]],
            'dropout_rate': [0.1, 0.2],
            'return_sequences': [False]  # single-step prediction
}
    
    def build_tcn(nb_filters=32, kernel_size=2, dilations=[1, 2, 4], 
                 dropout_rate=0.1, return_sequences=False):
        inputs = Input(shape=(1, X_train.shape[2]))
        x = TCN(nb_filters=nb_filters, kernel_size=kernel_size, 
               dilations=dilations, dropout_rate=dropout_rate,
               return_sequences=return_sequences)(inputs)
        outputs = Dense(1)(x)
        model = Model(inputs, outputs)
        model.compile(optimizer="adam", loss="mse")
        return model
    
    best_params = None
    best_rmse = float('inf')

    for filters in param_grid['nb_filters']:
        for kernel in param_grid['kernel_size']:
            for dilations in param_grid['dilations']:
                for dropout in param_grid['dropout_rate']:
                    for return_seq in param_grid['return_sequences']:
                        model = build_tcn(filters, kernel, dilations, dropout, return_seq)
                        try:
                            model.fit(X_train, y_train, epochs=50, batch_size=16, verbose=0)
                            
                            # Walk-forward validation (predict one step at a time)
                            predictions = []
                            history_X = X_train.copy()
                            history_y = y_train.copy()
                            
                            for i in range(len(X_val)):
                                # Predict next step
                                X_next = X_val[i].reshape(1, 1, -1)
                                pred = model.predict(X_next, verbose=0).flatten()[0]
                                predictions.append(pred)
                                
                            # Calculate RMSE
                            if len(y_val) == len(predictions):
                                rmse = np.sqrt(np.mean((y_val.flatten() - predictions) ** 2))
                            else:
                                print(f"Shape mismatch: y_val ({y_val.shape}), preds ({len(predictions)})")
                                continue
                            
                            if rmse < best_rmse:
                                best_rmse = rmse
                                best_params = {
                                    'nb_filters': filters,
                                    'kernel_size': kernel,
                                    'dilations': dilations,
                                    'dropout_rate': dropout,
                                    'return_sequences': return_seq
                                }
                        except Exception as e:
                            print(f"Hyperparameter set failed: {str(e)}")

    if best_params is None:
        print(f"❌ No valid models for {product_id}")
        return None

    # Final model training and walk-forward prediction
    final_model = build_tcn(**best_params)
    final_model.fit(X_train, y_train, epochs=50, batch_size=16, verbose=0)
    predictions = []

    # FIXED: Create DataFrame with both features and target
    history_data = np.column_stack([X_train[:, 0, :], y_train.flatten()])
    history = pd.DataFrame(history_data, columns=feature_set + [target_col])
    history.index = range(len(history))
    
    for i in range(horizon):
        try:
            X_future_step = X_val[i].reshape(1, 1, -1)
            pred = final_model.predict(X_future_step, verbose=0).flatten()[0]
            predictions.append(pred)
            
            # Update training data
            new_row = X_val[i, 0, :].tolist()
            new_row.append(pred)
            new_df = pd.DataFrame([new_row], columns=feature_set + [target_col])
            history = pd.concat([history, new_df]).reset_index(drop=True)
            
            # Retrain on expanded window
            rolling_window = 36 if len(history) > 36 else len(history)
            train_subset = history.iloc[-rolling_window:]
            
            X_train_update = train_subset[feature_set].values.reshape(-1, 1, len(feature_set))
            y_train_update = train_subset[target_col].values.reshape(-1, 1)
            
            final_model.fit(X_train_update, y_train_update, epochs=5, batch_size=16, verbose=0)
        
        except Exception as e:
            print(f"Iteration {i+1} failed: {str(e)}")
            predictions.append(predictions[-1] if predictions else y_train[-1][0])
    
    metrics = calculate_metrics(y_true=y_val.flatten(), y_pred=predictions, y_train=y_train.flatten())
    
    return {
        'product_id': product_id,
        'winsorize': winsorize,
        'features': feature_set,
        'metrics': metrics,
        'method': 'walkforward',
        'best_params': best_params,
        'validation_predictions': predictions
    }

def find_best_tcn_config(product_id, df_train, lagged_df=None, target_col="Sales", 
                            n_jobs=-1, custom_param_grids=None):
    # Validate input data
    if not isinstance(df_train.index, pd.DatetimeIndex):
        raise TypeError("❌ df_train index is not a DatetimeIndex!")
    if df_train.index.isnull().any():
        raise ValueError("❌ Some index values could not be parsed into datetime!")
    
    results = []
    data = df_train.copy()
    
    # Feature groups for original data
    normal_features = [col for col in data.columns if col != target_col]
    normal_feature_combinations = generate_feature_combinations(normal_features, max_features=6)  
    normal_feature_combinations = normal_feature_combinations
    
    # Test original data configurations
    for winsorize in [True, False]:
        for features in normal_feature_combinations:
            try:
                print(f"\nTesting - Winsorize: {winsorize}, Features: {features}")

                # Use custom param grid if provided for this feature set
                param_grid = None
                if custom_param_grids and tuple(features) in custom_param_grids:
                    param_grid = custom_param_grids[tuple(features)]

                result = choose_parameters_tcn(product_id=product_id,df_train=data,feature_set=features,
                    target_col=target_col,winsorize=winsorize,n_jobs=n_jobs,param_grid=param_grid)
                
                if result:
                    print(f"✅ Success - RMSE: {result['metrics']['RMSE']:.2f}")
                    results.append(result)
            except Exception as e:
                print(f"⚠️ Configuration failed: {str(e)}")
                continue
    
    # Test lagged data configurations if available
    if lagged_df is not None:
        lagged_data = lagged_df.copy()
        lag_features = [col for col in lagged_data.columns if 'lag' in col]
        ma_features = [col for col in lagged_data.columns if '_ma_' in col]
        macro_features = [col for col in data.columns if col != target_col]
        
        # Generate lag feature combinations
        lag_feature_combinations = generate_feature_combinations(lag_features, max_features=5)
        
        # Create combined feature sets
        full_combinations = []
        for lag_combo in lag_feature_combinations:
            full_combinations.append(lag_combo)  # Just lags
            full_combinations.append(lag_combo + macro_features)  # Lags + macros
            full_combinations.append(lag_combo + ma_features)  # Lags + MAs
            full_combinations.append(lag_combo + macro_features + ma_features)  # All
        
        # Remove duplicates
        seen = set()
        unique_combinations = []
        for combo in full_combinations:
            combo_tuple = tuple(sorted(combo))
            if combo_tuple not in seen:
                seen.add(combo_tuple)
                unique_combinations.append(combo)
        
        # Test all unique combinations
        for winsorize in [True, False]:
            for features in unique_combinations:
                try:
                    print(f"\nTesting - Lagged Features: {features}")

                    # Use custom param grid if provided for this feature set
                    param_grid = None
                    if custom_param_grids and tuple(features) in custom_param_grids:
                        param_grid = custom_param_grids[tuple(features)]

                    result = choose_parameters_tcn(product_id=product_id,df_train=lagged_data,feature_set=features,
                                                       target_col=target_col,winsorize=winsorize,n_jobs=n_jobs,param_grid=param_grid)
                    
                    if result:
                        print(f"✅ Success - RMSE: {result['metrics']['RMSE']:.2f}")
                except Exception as e:
                    print(f"⚠️ Lagged configuration failed: {str(e)}")
                    continue
    
    if not results:
        raise ValueError(f"No valid configurations for {product_id}")
    
    # Select best configuration by RMSE
    best_config = min(results, key=lambda x: x['metrics']['RMSE'])
    
    print(f"\n✅ Best configuration for {product_id}:")
    print(f"- Winsorize: {best_config['winsorize']}")
    print(f"- Features: {best_config['features']}")
    print(f"- Best params: {best_config['best_params']}")
    print(f"- RMSE: {best_config['metrics']['RMSE']:.2f}")
    print(f"- Method: {best_config.get('method', 'walkforward')}")
    
    return best_config, results

def process_product_parallel(product_id, custom_param_grids=None):
    try:
        df_train = product_dfs[product_id].rename(columns={product_id: "Sales"})
        
        # Ensure DatetimeIndex
        if not isinstance(df_train.index, pd.DatetimeIndex):
            df_train.index = pd.to_datetime(df_train.index, errors="coerce")
        
        if df_train.index.isnull().any():
            raise ValueError("❌ Some index values could not be parsed into datetime!")

        lagged_df = lagged_product_dfs.get(product_id, None)
        best_config, results = find_best_tcn_config(product_id=product_id,df_train=df_train,lagged_df=lagged_df,target_col="Sales", custom_param_grids=custom_param_grids)
        
        return best_config, results
    except Exception as e:
        print(f"⚠️ Failed to process {product_id}: {str(e)}")
        return None

In [ ]:
all_product_ids = set(product_dfs.keys())
all_results_tcn = []  # List to store all results
best_configs_tcn = []  # List to store best configurations

for product_id in all_product_ids:
    print(f"Processing product: {product_id}")
    
    try:
        best_config_tcn, results_tcn = process_product_parallel(product_id, custom_param_grids=None)
        
        if best_config_tcn is not None:
            best_configs_tcn.append(best_config_tcn)  # Store best config
        else:
            print(f"Warning: No best config returned for product {product_id}")
        
        if results_tcn is not None:
            all_results_tcn.extend(results_tcn)  # Store results
        else:
            print(f"Warning: No results returned for product {product_id}")
            
    except Exception as e:
        print(f"Error processing product {product_id}: {str(e)}")

Processing product: P14

Testing - Winsorize: True, Features: ['PRO27392_org']
Preparing data...
Sales: Winsorized Bounds -> Lower = -2064.76, Upper = 73222.25
✅ Success - RMSE: 14978.79

Testing - Winsorize: True, Features: ['PRO28380_org']
Preparing data...
Sales: Winsorized Bounds -> Lower = -2064.76, Upper = 73222.25
✅ Success - RMSE: 15484.21

Testing - Winsorize: True, Features: ['PRO27756_org']
Preparing data...
Sales: Winsorized Bounds -> Lower = -2064.76, Upper = 73222.25
✅ Success - RMSE: 16197.10

Testing - Winsorize: True, Features: ['PRO27392_org', 'PRO28380_org']
Preparing data...
Sales: Winsorized Bounds -> Lower = -2064.76, Upper = 73222.25
✅ Success - RMSE: 15385.45

Testing - Winsorize: True, Features: ['PRO27392_org', 'PRO27756_org']
Preparing data...
Sales: Winsorized Bounds -> Lower = -2064.76, Upper = 73222.25
✅ Success - RMSE: 15505.44

Testing - Winsorize: True, Features: ['PRO28380_org', 'PRO27756_org']
Preparing data...
Sales: Winsorized Bounds -> Lower = -206

#### 5. Arima

In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from pmdarima import auto_arima
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error

# Ensure time-series data is sorted and complete
df_p1_lags = df_p1_lags.sort_index()
df_p1_lags = df_p1_lags.asfreq('MS').interpolate()  # Fill missing months

# Fit Auto ARIMA Model (Using all data initially)
auto_model = auto_arima(
    df_p1_lags['P1'].dropna(),  
    exogenous=df_p1_lags[['P1_lag_1', 'P1_lag_2', 'P1_lag_3']].dropna(),  
    seasonal=False, 
    trace=True,
    stepwise=True,
    suppress_warnings=True,
    error_action='ignore',
    max_p=5, 
    max_q=5,
    d=None,  # Let ADF test decide differencing
    test='adf',  
    scoring='mse'
)
print(auto_model.summary())

# Walk-Forward Forecasting with Expanding Window
def walk_forward_forecast(model, df, horizon=10, initial_train_size=33):
    """
    Walk-forward validation using an expanding window approach.
    Predicts 'horizon' steps ahead at each iteration.
    """
    predictions, true_values, lower_bounds, upper_bounds = [], [], [], []

    for i in range(len(df) - initial_train_size - horizon + 1):
        train = df.iloc[: i + initial_train_size]  # Expanding training set
        test = df.iloc[i + initial_train_size : i + initial_train_size + horizon]

        model.fit(train['P1'])
        pred, conf_int = model.predict(n_periods=len(test), return_conf_int=True)

        if len(pred) == len(test):  # Ensure consistent shape
            predictions.append(pred)
            true_values.append(test['P1'].values)
            lower_bounds.append(conf_int[:, 0])
            upper_bounds.append(conf_int[:, 1])

    return np.array(true_values), np.array(predictions), np.array(lower_bounds), np.array(upper_bounds)

# Perform Walk-Forward Forecasting
true_values, predictions, lower_bounds, upper_bounds = walk_forward_forecast(auto_model, df_p1_lags, horizon=10)

# Compute Errors
def print_error(true, pred):
    RMSE = mean_squared_error(true.flatten(), pred.flatten(), squared=False)
    R2 = r2_score(true.flatten(), pred.flatten())
    MAPE = mean_absolute_percentage_error(true.flatten(), pred.flatten())
    print("RMSE:", RMSE)
    print("MAPE:", round(MAPE * 100, 2), "%")
    print("R2:", R2)

print_error(true_values, predictions)

# ✅ Plot Full Dataset + Predictions + Confidence Intervals
fig = go.Figure()

# Add the real data (Training + Test)
fig.add_trace(go.Scatter(
    x=df_p1_lags.index,
    y=df_p1_lags['P1'],
    mode='lines',
    name='Real Data',
    line=dict(color='black', width=2)
))

# Add predictions over time
for i in range(len(predictions)):
    forecast_index = df_p1_lags.index[i + 33 : i + 33 + 10]

    fig.add_trace(go.Scatter(
        x=forecast_index, 
        y=predictions[i],
        mode='lines',
        name=f'Prediction {i+1}',
        line=dict(dash='dash', width=1, color='blue')
    ))

    # Add confidence interval shading
    fig.add_trace(go.Scatter(
        x=list(forecast_index) + list(forecast_index[::-1]),
        y=list(upper_bounds[i]) + list(lower_bounds[i][::-1]),
        fill='toself',
        fillcolor='rgba(0,255,0,0.2)',
        line=dict(color='rgba(255,255,255,0)'),
        name='Confidence Interval',
        showlegend=(i == 0)
    ))

# Customize layout
fig.update_layout(
    title="Walk-Forward Forecasting (10-month prediction horizon)",
    xaxis_title="Months",
    yaxis_title="P1 Values",
    legend_title="Legend",
    template="plotly"
)

fig.show()


NameError: name 'df_p1_lags' is not defined

Com macros

In [ ]:
from pmdarima import auto_arima
from pmdarima.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score
import warnings
warnings.filterwarnings('ignore')


def choose_parameters_arima(product_id, df_train, target_col='Sales', winsorize=False, feature_set=None, horizon=10, seasonal=False, stationary=False):
    """
    Optimized function to prepare data, run auto_arima, and evaluate performance.
    """
    # Common Preprocessing
    X_train, X_val, y_train, y_val = prepare_time_series_data(df_train, product_id, target_col, feature_set, horizon, winsorize)

    # Convert to numpy arrays
    y_train, y_val = y_train.values, y_val.values
    X_train, X_val = X_train.values if not X_train.empty else None, X_val.values if not X_val.empty else None

    # Auto ARIMA Model
    model = auto_arima(y=y_train, X=X_train, stationary=stationary, seasonal=seasonal, suppress_warnings=True, stepwise=True, error_action='ignore', trace=False)

    # Walk-forward Validation
    predictions = []
    for i in range(len(y_val)):
        new_x = X_val[i].reshape(1, -1) if X_val is not None else None
        model.update([y_val[i]], X=new_x)
        pred = model.predict(n_periods=1, X=new_x)[0] if new_x is not None else model.predict(n_periods=1)[0]
        predictions.append(pred)

    # Evaluate Model
    metrics = calculate_metrics(y_val, predictions)

    return {
        'product_id': product_id,
        'winsorize': winsorize,
        'features': feature_set if feature_set else 'all',
        'best_params': {'order': model.order, 'seasonal_order': model.seasonal_order if seasonal else None, 'aic': model.aic(), 'bic': model.bic()},
        'metrics': metrics,
        'model': model}


def find_best_arima_config(product_id, df_train, target_col='Sales'):
    """
    Find optimal ARIMA configuration matching original testing approach
    """
    data = df_train[product_id].copy()
    available_features = [col for col in data.columns if col != target_col]
    
    # Generate feature combinations (max 3 for ARIMAX stability)
    feature_combinations = generate_feature_combinations(available_features, max_features=3)
    
    # Test all configurations - consistent with original approach
    results = []
    for winsorize in [True, False]:
        for features in feature_combinations:
            for seasonal in [False, True]:  # Test both seasonal and non-seasonal
                print(f"\nTesting config - Winsorize: {winsorize}, Features: {features}, Seasonal: {seasonal}")
                
                result = choose_parameters_arima(
                    product_id=product_id,
                    df_train=df_train,
                    target_col=target_col,
                    winsorize=winsorize,
                    feature_set=features,
                    seasonal=seasonal
                )
                results.append(result)
    
    # Find best configuration (lowest RMSE)
    best_config = min(results, key=lambda x: x['metrics']['RMSE'])

    print(f"\n✅ Best configuration for product {product_id}:")
    print(f"- Winsorize: {best_config['winsorize']}")
    print(f"- Features: {best_config['features']}")
    print(f"- Seasonal: {best_config['best_params']['seasonal_order'] is not None}")
    print(f"- Order (p,d,q): {best_config['best_params']['order']}")
    print(f"- RMSE: {best_config['metrics']['RMSE']:.2f}")
    print(f"- R²: {best_config['metrics']['R2']:.4f}")
    
    return best_config

def process_product_arima(product_id):
    """
    Consistent wrapper function matching original format
    """
    best_config = find_best_arima_config(
        product_id=product_id,
        df_train=df_train,
        target_col='Sales'
    )

    return {
        'Product_ID': product_id,
        'Model': 'ARIMA',
        'Winsorize': best_config['winsorize'],
        'Features': best_config['features'],
        'Order': best_config['best_params']['order'],
        'Seasonal_Order': best_config['best_params']['seasonal_order'],
        'RMSE': best_config['metrics']['RMSE'],
        'R²': best_config['metrics']['R2']
    }

#### 6. NeuralProphet

In [ ]:
def choose_parameters_neuralprophet(product_id, df_train, feature_set, target_col="Sales", 
                                  param_grid=None, n_jobs=-1, winsorize=False, horizon=7):
    """Optimized NeuralProphet forecasting with walk-forward validation"""
    
    if len(df_train) < 12:
        print(f"⚠️ Skipping {product_id} - only {len(df_train)} observations")
        return None
    
    print('Preparing data...')
    try:
        X_train, X_val, y_train, y_val = prepare_time_series_data(
            df_train, feature_set, target_col, horizon, winsorize)
        
        assert X_train.index[-1] < X_val.index[0], "Validation data must be after training data"
        
        # Prepare data for NeuralProphet (requires 'ds' and 'y' columns)
        train_df = pd.DataFrame({
            'ds': X_train.index,
            'y': y_train
        })
        
        val_df = pd.DataFrame({
            'ds': X_val.index,
            'y': y_val
        })
        
        # Add features if present
        if feature_set:
            for feature in feature_set:
                train_df[feature] = X_train[feature].values
                val_df[feature] = X_val[feature].values
        
    except Exception as e:
        print(f"Data preparation failed: {str(e)}")
        return None

    if param_grid is None:
        param_grid = {
            'n_lags': [3, 7, 14],
            'n_forecasts': [horizon],
            'changepoints_range': [0.8, 0.95],
            'num_hidden_layers': [0, 1],
            'd_hidden': [32, 64],
            'learning_rate': [0.01, 0.001],
            'batch_size': [16, 32],
            'epochs': [50, 100]
        }
    
    best_params = None
    best_rmse = float('inf')

    # Grid search over parameter combinations
    for n_lags in param_grid['n_lags']:
        for n_forecasts in param_grid['n_forecasts']:
            for changepoints_range in param_grid['changepoints_range']:
                for num_hidden_layers in param_grid['num_hidden_layers']:
                    for d_hidden in param_grid['d_hidden']:
                        for learning_rate in param_grid['learning_rate']:
                            for batch_size in param_grid['batch_size']:
                                for epochs in param_grid['epochs']:
                                    try:
                                        # Initialize model with current parameters
                                        model = NeuralProphet(
                                            n_lags=n_lags,n_forecasts=n_forecasts,changepoints_range=changepoints_range,num_hidden_layers=num_hidden_layers,d_hidden=d_hidden,
                                            learning_rate=learning_rate,normalize='soft')
                                        
                                        # Add features if present
                                        if feature_set:
                                            for feature in feature_set:
                                                model = model.add_lagged_regressor(feature)
                                        
                                        # Train model
                                        metrics = model.fit(train_df, freq='D', 
                                                          batch_size=batch_size, 
                                                          epochs=epochs,
                                                          silent=True)
                                        
                                        # Walk-forward validation
                                        predictions = []
                                        history = train_df.copy()
                                        
                                        for i in range(len(val_df)):
                                            # Create future dataframe with known regressors
                                            future = model.make_future_dataframe(
                                                history, 
                                                periods=1,
                                                n_historic=len(history))
                                            
                                            if feature_set:
                                                # Add future regressor values
                                                for feature in feature_set:
                                                    future[feature] = val_df[feature].iloc[i]
                                            
                                            # Predict next step
                                            forecast = model.predict(future)
                                            pred = forecast['yhat1'].iloc[0]
                                            predictions.append(pred)
                                            
                                            # Update history with actual observation
                                            new_row = val_df.iloc[i].to_frame().T
                                            history = pd.concat([history, new_row])
                                        
                                        # Calculate RMSE
                                        if len(y_val) == len(predictions):
                                            rmse = np.sqrt(np.mean((y_val - predictions) ** 2))
                                        else:
                                            print(f"Shape mismatch: y_val ({len(y_val)}), preds ({len(predictions)})")
                                            continue
                                        
                                        if rmse < best_rmse:
                                            best_rmse = rmse
                                            best_params = {
                                                'n_lags': n_lags,
                                                'n_forecasts': n_forecasts,
                                                'changepoints_range': changepoints_range,
                                                'num_hidden_layers': num_hidden_layers,
                                                'd_hidden': d_hidden,
                                                'learning_rate': learning_rate,
                                                'batch_size': batch_size,
                                                'epochs': epochs
                                            }
                                        
                                    except Exception as e:
                                        print(f"Hyperparameter set failed: {str(e)}")
                                        continue

    if best_params is None:
        print(f"❌ No valid models for {product_id}")
        return None

    # Train final model with best parameters
    final_model = NeuralProphet(
        n_lags=best_params['n_lags'],
        n_forecasts=best_params['n_forecasts'],
        changepoints_range=best_params['changepoints_range'],
        num_hidden_layers=best_params['num_hidden_layers'],
        d_hidden=best_params['d_hidden'],
        learning_rate=best_params['learning_rate'],
        normalize='soft'
    )
    
    if feature_set:
        for feature in feature_set:
            final_model = final_model.add_lagged_regressor(feature)
    
    # Train on full training data
    final_model.fit(train_df, freq='D', 
                   batch_size=best_params['batch_size'],
                   epochs=best_params['epochs'],
                   silent=True)
    
    # Make predictions on validation set
    predictions = []
    history = train_df.copy()
    
    for i in range(horizon):
        try:
            # Create future dataframe
            future = final_model.make_future_dataframe(
                history, 
                periods=1,
                n_historic=len(history)
            
            if feature_set:
                # Add future regressor values if available
                if i < len(val_df):
                    for feature in feature_set:
                        future[feature] = val_df[feature].iloc[i]
                else:
                    # If beyond validation data, use last known values
                    for feature in feature_set:
                        future[feature] = history[feature].iloc[-1]
            
            # Predict next step
            forecast = final_model.predict(future)
            pred = forecast['yhat1'].iloc[0]
            predictions.append(pred)
            
            # Update history with predicted observation
            if i < len(val_df):
                new_row = val_df.iloc[i].to_frame().T
            else:
                # If beyond validation data, create synthetic row
                new_row = pd.DataFrame({
                    'ds': [history['ds'].iloc[-1] + pd.Timedelta(days=1)],
                    'y': pred
                })
                if feature_set:
                    for feature in feature_set:
                        new_row[feature] = history[feature].iloc[-1]
            
            history = pd.concat([history, new_row])
            
        except Exception as e:
            print(f"Iteration {i+1} failed: {str(e)}")
            predictions.append(predictions[-1] if predictions else train_df['y'].iloc[-1])
    
    metrics = calculate_metrics(y_true=y_val, y_pred=predictions[:len(y_val)], 
                              y_train=train_df['y'].values)
    
    return {
        'product_id': product_id,
        'winsorize': winsorize,
        'features': feature_set,
        'metrics': metrics,
        'method': 'walkforward',
        'best_params': best_params,
        'validation_predictions': predictions
    }

#### Prediction for he Macro Features used

In [ ]:
# Define the function to classify variables
def classify_variable(series):
    """Classifies variable based on normality and stationarity tests."""
    
    # Remove NaN values for testing
    clean_series = series.dropna()

    # Check for normality
    if len(clean_series) > 3:
        stat, p_value = shapiro(clean_series)
        is_normal = p_value > 0.05  # p-value > 0.05 means normal
    else:
        is_normal = False  # Not enough data to test normality

    # Check for stationarity
    if len(clean_series) > 3:
        adf_stat, adf_p_value, _, _, _, _ = adfuller(clean_series)
        is_stationary = adf_p_value < 0.05  # p-value < 0.05 means stationary
    else:
        is_stationary = False  # Not enough data

    return is_normal, is_stationary

# Function to automatically fill missing values
def auto_impute_missing_values(df_train, df_test):
    """Automatically selects the best imputation method for each missing variable."""
    
    # Identify missing columns in test set
    missing_columns = df_test.columns[df_test.isnull().any()]
    
    # Iterate through missing columns
    for col in missing_columns:
        print(f"Processing: {col}")

        series = df_train[col]  # Use train data for imputation
        is_normal, is_stationary = classify_variable(series)

        if is_normal:
            # Case 1: Normally distributed → Sample from normal distribution
            print(f" - {col} is normal → Using Mean & Std Sampling")
            mean_value, std_value = series.mean(), series.std()
            num_missing = df_test[col].isnull().sum()
            predictions = norm.rvs(loc=mean_value, scale=std_value, size=num_missing)
        
        elif is_stationary:
            # Case 2: Stationary but non-normal → Simple Exponential Smoothing
            print(f" - {col} is stationary → Using Simple Exponential Smoothing")
            model = SimpleExpSmoothing(series.dropna()).fit()
            predictions = model.forecast(steps=df_test[col].isnull().sum())

        elif not is_stationary:
            # Case 3: Non-Stationary → ARIMA
            print(f" - {col} is non-stationary → Using ARIMA")
            model = ARIMA(series.dropna(), order=(1, 1, 1))  # (p,d,q) chosen based on domain knowledge
            fitted_model = model.fit()
            predictions = fitted_model.forecast(steps=df_test[col].isnull().sum())

        else:
            # Case 4: If nothing works → Use XGBoost Regression
            print(f" - {col} is complex → Using XGBoost Regression")
            train_data = df_train.dropna(subset=[col])  # Drop missing values for training
            X_train = train_data.drop(columns=[col])  # Exclude target column
            y_train = train_data[col]  # Target column

            X_test = df_test.loc[df_test[col].isnull(), X_train.columns]  # Only missing values

            model = XGBRegressor(n_estimators=100, learning_rate=0.1)
            model.fit(X_train, y_train)
            predictions = model.predict(X_test)

        # Assign predictions
        missing_indexes = df_test[df_test[col].isnull()].index
        df_test.loc[missing_indexes, col] = predictions

    return df_test

# Example usage
df_train = remerged_data[1]  # Use remerged train data
df_test = test_1.copy()  # Copy test set

# Apply automatic imputation
df_test_filled = auto_impute_missing_values(df_train, df_test)

# Check results
print(df_test_filled.head())